In [34]:
# imports
import os
import pandas as pd
import tensorflow as tf




In [79]:
from keras.src.utils import load_img, img_to_array

# load data
basefolder = 'C:\\Users\\walra\\Stichting Hogeschool Utrecht\\Data Sience MNLE - General\\pools\\Beeldmateriaal_ortho_2023'

# create multichannel dataset
# load base rgb images
folder='rgb8'
# create dataframe with all filenames from folder
filenames_rgb = []
for root, dirs, files in os.walk(os.path.join(basefolder, folder)):
    for file in files:
        if file.endswith('.png'):
            filenames_rgb.append(file)

# load infrared images
folder='infrared'
# create dataframe with all filenames from folder
filenames_infrared = []
for root, dirs, files in os.walk(os.path.join(basefolder, folder)):
    for file in files:
        if file.endswith('.png'):
            filenames_infrared.append(file)
            
# make new list with filenames that are in both lists
filenames = list(set(filenames_rgb) & set(filenames_infrared))

# load labels from csv
labels = pd.read_csv(os.path.join(basefolder, 'labels.csv'))
labels = labels.query('filename.isin(@filenames)')
labels = labels.set_index('filename')
labels = labels.groupby('filename')


# Create multichannel dataset

In [80]:
batch_images = []
for group in labels.groups:
    images = []
    for channel in ['rgb8', 'infrared']:
        img_path = os.path.join(basefolder, channel, group)
        img = load_img(img_path, color_mode='rgb', target_size=(1600, 1600))
        img = img_to_array(img)
        images.append(img)
    
    concatenated = tf.concat(images, axis=-1)
    batch_images.append(concatenated)

batch_images

[<tf.Tensor: shape=(1600, 1600, 6), dtype=float32, numpy=
 array([[[ 39.,  50.,  56., 197., 114., 130.],
         [ 41.,  52.,  58., 197., 114., 130.],
         [ 41.,  52.,  58., 197., 114., 130.],
         ...,
         [ 35.,  41.,  55., 161., 195., 179.],
         [ 35.,  41.,  55., 161., 195., 179.],
         [ 35.,  41.,  55., 161., 195., 179.]],
 
        [[ 45.,  56.,  62., 197., 114., 130.],
         [ 36.,  47.,  53., 197., 114., 130.],
         [ 29.,  40.,  46., 197., 114., 130.],
         ...,
         [ 35.,  41.,  55., 161., 195., 179.],
         [ 35.,  41.,  55., 161., 195., 179.],
         [ 35.,  41.,  55., 161., 195., 179.]],
 
        [[ 45.,  56.,  60., 197., 114., 130.],
         [ 37.,  48.,  52., 197., 114., 130.],
         [ 29.,  40.,  44., 197., 114., 130.],
         ...,
         [ 35.,  41.,  55., 161., 195., 179.],
         [ 35.,  41.,  55., 161., 195., 179.],
         [ 35.,  41.,  55., 161., 195., 179.]],
 
        ...,
 
        [[ 44.,  58.,  61., 19

In [82]:
# tile the images in 3x3
tiled_images = tf.tile(batch_images, multiples=[3, 3])

InvalidArgumentError: {{function_node __wrapped__Tile_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expected multiples argument to be a vector of length 4 but got length 2 [Op:Tile]